In [76]:
import pandas as pd
from multiprocessing import Pool, cpu_count


In [2]:
store = pd.HDFStore('/home/kelleyld/OTB.h5')

In [3]:
games = store.get("games")
games.head()

,GameID,Outcome,WhiteName,BlackName,WhiteElo,BlackElo,Date
0,1,w,"îwicarz, Maciej","Grossmann, Ryszard",NaN,2205.0,2005-07-01
1,2,b,"îwicarz, Maciej","Starostits, Ilmars",NaN,2450.0,2005-06-29
2,3,b,"OEwicarz, Maciej","Zawadzki, Stanis=/+aw",NaN,NaN,2005-07-02
3,4,w,"Åkesson, Joel","Berg, Emanuel",2333.0,2495.0,2004-01-12
4,5,s,"Åkesson, Joel","Cioara, Andrei-Nestor",2360.0,2383.0,2004-06-20


In [4]:
openings = store.get("openings")
opening_results = store.get("opening_results")

In [5]:
opening_results.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency
491416,P,4,1,3,4,p,2,6,4,2,...,2,p,0,6,5,0,0.306617,0.390478,0.302905,146290
594900,P,4,1,3,4,p,4,6,4,4,...,6,b,5,7,6,4,0.305951,0.392637,0.301412,80405
491421,P,4,1,3,4,p,2,6,4,2,...,2,p,6,6,5,6,0.307274,0.390700,0.302026,57164
480899,P,4,1,3,4,p,2,6,4,2,...,2,p,4,6,4,4,0.305496,0.388587,0.305917,45143
274089,P,3,1,3,3,n,6,7,5,5,...,5,k,4,0,0,6,0.308641,0.390539,0.300820,41942


In [6]:
openings.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Outcome
0,P,4,1,3,4,p,3,6,4,3,...,5,0,3,2,n,6,7,5,5,NaN
1,P,4,1,3,4,p,2,6,4,2,...,5,0,4,1,b,2,7,6,3,w
2,N,6,0,2,5,p,3,6,4,3,...,1,0,1,3,n,6,7,5,5,b
3,P,3,1,3,3,n,6,7,5,5,...,1,0,2,2,p,3,6,4,3,b
4,P,3,1,3,3,p,4,6,5,4,...,0,1,2,0,b,1,3,1,3,w


In [7]:
moves = store.get('moves')
basic_moves = moves.loc[:, ['GameID', 
                            'Ply', 
                            'MovedPiece',
                            'SourceCol', 
                            'SourceRow', 
                            'DestRow', 
                            'DestCol']]
opening_moves = [basic_moves[basic_moves['Ply'] == ply] for ply in range(1, 11)]
for opening_move in opening_moves: 
    opening_move.drop('Ply', axis=1, inplace=True)
for index, opening_move in enumerate(opening_moves):
    suffix = str(index + 1)
    opening_move.columns = ['GameID', 
                            'MovedPiece' + suffix, 
                            'SourceCol' + suffix, 
                            'SourceRow' + suffix, 
                            'DestRow' + suffix, 
                            'DestCol' + suffix]
partial_book = opening_moves[0]
for index, opening_move in enumerate(opening_moves[1:]):
    partial_book = partial_book.merge(opening_move, how='outer', on='GameID')
opening_book = partial_book.dropna()
opening_book.head()

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,GameID,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,MovedPiece9,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10
0,1,P,4,1,3,4,p,3,6,4,...,B,5,0,3,2,n,6,7,5,5
1,2,P,4,1,3,4,p,2,6,4,...,B,5,0,4,1,b,2,7,6,3
2,3,N,6,0,2,5,p,3,6,4,...,N,1,0,1,3,n,6,7,5,5
3,4,P,3,1,3,3,n,6,7,5,...,N,1,0,2,2,p,3,6,4,3
4,5,P,3,1,3,3,p,4,6,5,...,P,0,1,2,0,b,1,3,1,3


In [8]:
opening_results.columns

Index(['MovedPiece1', 'SourceCol1', 'SourceRow1', 'DestRow1', 'DestCol1',
       'MovedPiece2', 'SourceCol2', 'SourceRow2', 'DestRow2', 'DestCol2',
       'MovedPiece3', 'SourceCol3', 'SourceRow3', 'DestRow3', 'DestCol3',
       'MovedPiece4', 'SourceCol4', 'SourceRow4', 'DestRow4', 'DestCol4',
       'MovedPiece5', 'SourceCol5', 'SourceRow5', 'DestRow5', 'DestCol5',
       'MovedPiece6', 'SourceCol6', 'SourceRow6', 'DestRow6', 'DestCol6',
       'MovedPiece7', 'SourceCol7', 'SourceRow7', 'DestRow7', 'DestCol7',
       'MovedPiece8', 'SourceCol8', 'SourceRow8', 'DestRow8', 'DestCol8',
       'MovedPiece9', 'SourceCol9', 'SourceRow9', 'DestRow9', 'DestCol9',
       'MovedPiece10', 'SourceCol10', 'SourceRow10', 'DestRow10', 'DestCol10',
       'Black Wins', 'White Wins', 'Stalemates', 'Frequency'],
      dtype='object')

In [9]:
merge_cols = []
for c in opening_book.columns[1:len(opening_book.columns)].values:
    merge_cols.append(c)
merge_cols

['MovedPiece1',
 'SourceCol1',
 'SourceRow1',
 'DestRow1',
 'DestCol1',
 'MovedPiece2',
 'SourceCol2',
 'SourceRow2',
 'DestRow2',
 'DestCol2',
 'MovedPiece3',
 'SourceCol3',
 'SourceRow3',
 'DestRow3',
 'DestCol3',
 'MovedPiece4',
 'SourceCol4',
 'SourceRow4',
 'DestRow4',
 'DestCol4',
 'MovedPiece5',
 'SourceCol5',
 'SourceRow5',
 'DestRow5',
 'DestCol5',
 'MovedPiece6',
 'SourceCol6',
 'SourceRow6',
 'DestRow6',
 'DestCol6',
 'MovedPiece7',
 'SourceCol7',
 'SourceRow7',
 'DestRow7',
 'DestCol7',
 'MovedPiece8',
 'SourceCol8',
 'SourceRow8',
 'DestRow8',
 'DestCol8',
 'MovedPiece9',
 'SourceCol9',
 'SourceRow9',
 'DestRow9',
 'DestCol9',
 'MovedPiece10',
 'SourceCol10',
 'SourceRow10',
 'DestRow10',
 'DestCol10']

In [10]:
results_with_id = pd.merge(opening_book, opening_results, how="left", right_on=merge_cols, left_on=merge_cols)
results_with_id.head()

,GameID,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency
0,1,P,4,1,3,4,p,3,6,4,...,2,n,6,7,5,5,0.288575,0.411072,0.300353,849.0
1,2,P,4,1,3,4,p,2,6,4,...,1,b,2,7,6,3,0.303108,0.395546,0.301346,6242.0
2,3,N,6,0,2,5,p,3,6,4,...,3,n,6,7,5,5,0.500000,0.000000,0.500000,2.0
3,4,P,3,1,3,3,n,6,7,5,...,2,p,3,6,4,3,0.316317,0.392890,0.290793,1097.0
4,5,P,3,1,3,3,p,4,6,5,...,0,b,1,3,1,3,0.325000,0.325000,0.350000,120.0


In [11]:
games.head()

,GameID,Outcome,WhiteName,BlackName,WhiteElo,BlackElo,Date
0,1,w,"îwicarz, Maciej","Grossmann, Ryszard",NaN,2205.0,2005-07-01
1,2,b,"îwicarz, Maciej","Starostits, Ilmars",NaN,2450.0,2005-06-29
2,3,b,"OEwicarz, Maciej","Zawadzki, Stanis=/+aw",NaN,NaN,2005-07-02
3,4,w,"Åkesson, Joel","Berg, Emanuel",2333.0,2495.0,2004-01-12
4,5,s,"Åkesson, Joel","Cioara, Andrei-Nestor",2360.0,2383.0,2004-06-20


# Example Run with Specific Player

In [12]:
joel_games = games.loc[games["WhiteName"] == "Åkesson, Joel"]

In [13]:
joel_results = results_with_id.loc[results_with_id.GameID.isin(joel_games.values)]
joel_results.head()

/opt/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,GameID,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency
3,4,P,3,1,3,3,n,6,7,5,...,2,p,3,6,4,3,0.316317,0.392890,0.290793,1097.0
4,5,P,3,1,3,3,p,4,6,5,...,0,b,1,3,1,3,0.325000,0.325000,0.350000,120.0
5,6,P,3,1,3,3,p,3,6,4,...,5,n,1,7,6,3,0.290182,0.398194,0.311623,6203.0
6,7,P,3,1,3,3,n,6,7,5,...,6,p,2,6,4,2,0.374126,0.367133,0.258741,286.0
7,8,P,3,1,3,3,n,6,7,5,...,6,p,4,6,4,4,0.428571,0.428571,0.142857,14.0


In [14]:
joel_games

,GameID,Outcome,WhiteName,BlackName,WhiteElo,BlackElo,Date
3,4,w,"Åkesson, Joel","Berg, Emanuel",2333.0,2495.0,2004-01-12
4,5,s,"Åkesson, Joel","Cioara, Andrei-Nestor",2360.0,2383.0,2004-06-20
5,6,s,"Åkesson, Joel","Gagarin, Vasilij",2360.0,2422.0,2004-06-17
6,7,w,"Åkesson, Joel","Nemeth, Sebastian",2360.0,2124.0,2004-06-13
7,8,w,"Åkesson, Joel","Sagit, Rauan",2360.0,2081.0,2004-06-15


# Full Run

## Get Players who have played the threshold number of games

In [103]:
GAME_THRESHOLD = 1000

In [104]:
games = games.dropna()

In [105]:
games_by_whitename = games.groupby("WhiteName")["GameID"].count()
games_by_whitename = games_by_whitename.drop("?")
games_by_whitename = games_by_whitename.loc[games_by_whitename >= GAME_THRESHOLD]

In [106]:
games_by_blackname = games.groupby("BlackName")["GameID"].count()
games_by_blackname = games_by_blackname.drop("?")
games_by_blackname = games_by_blackname.loc[games_by_blackname >= GAME_THRESHOLD]

In [107]:
games_count_per_player = games_by_whitename.append(games_by_blackname)
games_count_per_player.shape

(43,)

## Get qualified player games

In [108]:
qualified_player_names = []
for n in games_count_per_player.keys().values:
    qualified_player_names.append(n)

In [109]:
qualified_white = games.loc[games["WhiteName"].isin(qualified_player_names)]
qualified_black = games.loc[games["BlackName"].isin(qualified_player_names)]

In [110]:
qualified_white = qualified_white[["GameID", "Outcome", "WhiteName"]]
qualified_white.columns = ["GameID", "Outcome", "Name"]
qualified_white["Color"] = "white"

In [111]:
qualified_black = qualified_black[["GameID", "Outcome", "BlackName"]]
qualified_black.columns = ["GameID", "Outcome", "Name"]
qualified_black["Color"] = "black"

In [112]:
qualified_games = pd.concat([qualified_white, qualified_black])
qualified_games.head()

,GameID,Outcome,Name,Color
30906,31892,s,"Adams, Michael",white
30907,31893,w,"Adams, Michael",white
30917,31904,s,"Adams, Michael",white
30918,31905,w,"Adams, Michael",white
30922,31909,w,"Adams, Michael",white


In [113]:
qualified_games.shape

(60663, 4)

In [114]:
qualifed_merge = pd.merge(results_with_id, qualified_games, on="GameID")
qualifed_merge.head(10)

,GameID,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency,Outcome,Name,Color
0,1562,P,4,1,3,4,p,2,6,4,...,6,5,0,0.306617,0.390478,0.302905,146290.0,b,"Gelfand, Boris",black
1,3762,P,3,1,3,3,n,6,7,5,...,3,6,4,0.305158,0.385387,0.309456,698.0,b,"Tiviakov, Sergei",black
2,4152,P,4,1,3,4,p,2,6,4,...,6,4,4,0.305496,0.388587,0.305917,45143.0,s,"Carlsen, Magnus",black
3,4180,P,3,1,3,3,p,3,6,4,...,6,5,7,0.310345,0.379010,0.310645,3335.0,s,"Dreev, Aleksey",black
4,4181,P,4,1,3,4,p,2,6,4,...,6,5,3,0.000000,1.000000,0.000000,1.0,b,"Dreev, Aleksey",black
5,7574,P,4,1,3,4,p,4,6,4,...,3,5,3,0.305430,0.385520,0.309050,2210.0,s,"Bacrot, Etienne",black
6,7673,P,3,1,3,3,p,3,6,4,...,7,6,4,1.000000,0.000000,0.000000,1.0,s,"Korneev, Oleg",black
7,7674,P,4,1,3,4,p,4,6,4,...,6,5,5,0.307239,0.386731,0.306030,6617.0,w,"Korneev, Oleg",black
8,8747,N,6,0,2,5,n,6,7,5,...,7,6,3,0.322275,0.390995,0.286730,422.0,b,"Van Wely, Loek",black
9,9938,P,3,1,3,3,p,3,6,4,...,6,5,0,0.316527,0.385154,0.298319,1428.0,s,"Karjakin, Sergey",black


In [115]:
cols = qualifed_merge.columns[1: len(qualifed_merge.columns) - 7]
cols = ["Name"] + cols.tolist()

In [118]:
a = qualifed_merge.groupby(cols)

In [140]:
def agg_results(results):
    opening_counter = results.shape[0]
    data = pd.DataFrame(results[cols])
    data["opening_counter"] = opening_counter
    return data

In [141]:
with Pool(cpu_count() // 4) as pool:
    mapped = pool.map(agg_results, [group for _, group in a])
dirty_outcomes = pd.concat(mapped)
dirty_outcomes.head()

,Name,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,opening_counter
29406,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,4,0,0,6,p,6,6,5,6,1
27933,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,1,0,1,3,p,4,6,5,4,1
29434,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,4,1,3,4,b,2,7,3,6,1
48893,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,4,p,3,6,4,3,2
48901,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,4,p,3,6,4,3,2


In [142]:
dirty_outcomes

,Name,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,opening_counter
29406,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,4,0,0,6,p,6,6,5,6,1
27933,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,1,0,1,3,p,4,6,5,4,1
29434,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,4,1,3,4,b,2,7,3,6,1
48893,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,4,p,3,6,4,3,2
48901,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,4,p,3,6,4,3,2
11688,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,6,p,3,6,4,3,4
16263,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,6,p,3,6,4,3,4
16264,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,6,p,3,6,4,3,4
45374,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,6,p,3,6,4,3,4
21379,"Adams, Michael",N,6,0,2,5,n,6,7,5,...,5,0,1,6,b,2,7,6,1,1
